In [1]:
%matplotlib inline
%run ../../path_import.py
from plots import *
from training import *
from eubo import *
from model_naive_natparam import *

import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
data_path = "../gmm_dataset_conjugate2k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
_, N, D = Data.shape
K = 3 ## number of clusters
## Model Parameters
MCMC_SIZE = 10
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 64
## Training Parameters
BATCH_SIZE = 20
NUM_EPOCHS = 3000
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'ag-idw-init-eta-kl_v2'

DEVICE = torch.device('cuda:1')

In [3]:
enc_eta, enc_z, optimizer = initialize(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE, LEARNING_RATE)

In [4]:
enc_eta.load_state_dict(torch.load('../weights/enc-eta-%s' + PATH))
enc_z.load_state_dict(torch.load('../weights/enc-z-%s' + PATH))

In [ ]:
PATH2 = 'ag-idw-init-eta-kl_v2-stage2'

train(Eubo_idw_init_eta, enc_eta, enc_z, optimizer, Data, K, NUM_EPOCHS, 30, 20, BATCH_SIZE, PATH2, CUDA, DEVICE)

epoch=0, EUBO=-208.117, ELBO=-218.820, ESS=12.837 (62s)
epoch=1, EUBO=-208.705, ELBO=-219.548, ESS=12.802 (62s)
epoch=2, EUBO=-208.559, ELBO=-219.319, ESS=12.785 (62s)
epoch=3, EUBO=-208.197, ELBO=-219.031, ESS=12.831 (62s)


In [ ]:
torch.save(enc_eta.state_dict(), '../weights/enc-eta-%s' + PATH)
torch.save(enc_z.state_dict(), '../weights/enc-z-%s' + PATH)

In [ ]:
BATCH_SIZE_TEST = 50
obs, q_eta, q_z = test(Eubo_idw_init_eta, enc_eta, enc_z, Data, K, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE)

In [ ]:
%time plot_samples(obs, q_eta, q_z, PATH)